In [1]:
import numpy as np
import torch
import pickle
import time
import os
%matplotlib inline
import matplotlib.pyplot as plt


In [2]:
if not os.path.isfile('superpixels.zip'):
    print('downloading..')
    !curl https://www.dropbox.com/s/y2qwa77a0fxem47/superpixels.zip?dl=1 -o superpixels.zip -J -L -k
    !unzip superpixels.zip -d ../
    # !tar -xvf superpixels.zip -C ../
else:
    print('File already downloaded')

File already downloaded


In [3]:
import os
 # go to root folder of the project
print(os.getcwd())

/Users/tuyiwen/Desktop/gli/datasets/mnist


In [4]:
import pickle

%load_ext autoreload
%autoreload 2

from data.superpixels import SuperPixDatasetDGL 

from data.data import LoadData
from torch.utils.data import DataLoader
from data.superpixels import SuperPixDataset

Using backend: pytorch


In [5]:
dataset = LoadData('MNIST') # 54s
trainset, valset, testset = dataset.train, dataset.val, dataset.test

[I] Loading dataset MNIST...
train, test, val sizes : 55000 10000 5000
[I] Finished loading.
[I] Data load time: 29.9557s


In [6]:
print(dataset)

In [7]:
trainset, valset, testset = dataset.train, dataset.val, dataset.test

In [8]:
print(dataset.train[0][0].edges[0][0]['feat'])


tensor([[0.1504]], dtype=torch.float16)


In [6]:
train_idx=np.arange(0,55000,1)
valid_idx=np.arange(55000,60000,1)
test_idx=np.arange(60000,70000,1)



In [10]:
task_data = {
    "train": train_idx,
    "val": valid_idx,
    "test": test_idx
}
np.savez_compressed("graph_classification_1.npz", **task_data)

In [18]:
np.array(dataset.train[0][0].edata['feat'])

552

In [19]:
from tqdm import tqdm
node_list = []
labels = []
edges = []
edge_feats = []
node_feats = []
num_nodes = 0
for g, label in tqdm(dataset.train):
    node_list.append(np.arange(g.num_nodes()) + num_nodes)  # All the nodes are considered in a single graph
    
    labels.append(label)
    edges.append(np.stack([g.edges()[0],g.edges()[1]]).T + num_nodes)
    edge_feats.append(np.array(g.edata['feat']))
    node_feats.append(np.array(g.ndata['feat']))
    num_nodes += g.num_nodes()
for g, label in tqdm(dataset.val):
    node_list.append(np.arange(g.num_nodes()) + num_nodes)  # All the nodes are considered in a single graph
    labels.append(label)
    edges.append(np.stack([g.edges()[0],g.edges()[1]]).T + num_nodes)
    edge_feats.append(np.array(g.edata['feat']))
    node_feats.append(np.array(g.ndata['feat']))
    num_nodes += g.num_nodes()
for g, label in tqdm(dataset.test):
    node_list.append(np.arange(g.num_nodes()) + num_nodes)  # All the nodes are considered in a single graph


    labels.append(label)
    edges.append(np.stack([g.edges()[0],g.edges()[1]]).T + num_nodes)
    edge_feats.append(np.array(g.edata['feat']))
    node_feats.append(np.array(g.ndata['feat']))
    num_nodes += g.num_nodes()

100%|██████████| 10000/10000 [00:02<00:00, 4883.00it/s]


In [20]:

_edges = np.concatenate(edges, axis=0)
_labels = np.stack(labels).squeeze()
_edge_feats = np.concatenate(edge_feats)
_node_feats = np.concatenate(node_feats)

In [26]:
import scipy.sparse as sparse
_node_list = sparse.csr_matrix((70000, num_nodes))

for i, indices in enumerate(node_list):
    _node_list[i, indices] = 1

/Users/tuyiwen/Library/Python/3.8/lib/python/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [27]:
data = {
    "node_feats": _node_feats,
    "graph_class": _labels,
    "edge": _edges,
    "edge_feats": _edge_feats,
    "node_list": _node_list,
}

In [28]:
np.savez_compressed("mnist.npz", **data)